[Learning Lukasiewicz Logic Fragments by Quadratic Programming](http://ecmlpkdd2017.ijs.si/papers/paperID223.pdf)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cvxpy as cp

# 1. Toy problem

---
メモ

* 各 p に対してデータの数が揃っている等の理由により，実装がシンプルになっているので，他の問題に直ちに適用することはできない

## 1.1 データの準備

In [2]:
L_1 = [(.1, .5, -1), (.4, .4, -1), (.3, .8, 1), (.9, .7, 1)]
L_2 = [(.1, .3, -1), (.6, .4, -1), (.2, .8, 1), (.7, .6, 1)]
L_3 = [(.4, .2, -1), (.9, .3, -1), (.2, .6, 1), (.5, .7, 1)]

U = [(.1, .5), (.3, .7), (.5, .4), (.8, .3), (.9, .2), (1, .5)] # U_1 = U_2 = U_3 = U ということでいいのか


display(L_1)
display(L_2)
display(L_3)
print()
display(U)

[(0.1, 0.5, -1), (0.4, 0.4, -1), (0.3, 0.8, 1), (0.9, 0.7, 1)]

[(0.1, 0.3, -1), (0.6, 0.4, -1), (0.2, 0.8, 1), (0.7, 0.6, 1)]

[(0.4, 0.2, -1), (0.9, 0.3, -1), (0.2, 0.6, 1), (0.5, 0.7, 1)]

[(0.1, 0.5), (0.3, 0.7), (0.5, 0.4), (0.8, 0.3), (0.9, 0.2), (1, 0.5)]

In [3]:
U = np.array(U).T

display(U)

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

In [4]:
x_L_1, y_L_1 = np.array([[x_l[0], x_l[1]] for x_l in L_1]).T, np.array([[y_l[2]] for y_l in L_1]).T
x_L_2, y_L_2 = np.array([[x_l[0], x_l[1]] for x_l in L_2]).T, np.array([[y_l[2]] for y_l in L_2]).T
x_L_3, y_L_3 = np.array([[x_l[0], x_l[1]] for x_l in L_3]).T, np.array([[y_l[2]] for y_l in L_3]).T

display(x_L_1)
display(y_L_1)
print()
display(x_L_2)
display(y_L_2)
print()
display(x_L_3)
display(y_L_3)

array([[0.1, 0.4, 0.3, 0.9],
       [0.5, 0.4, 0.8, 0.7]])

array([[-1, -1,  1,  1]])

array([[0.1, 0.6, 0.2, 0.7],
       [0.3, 0.4, 0.8, 0.6]])

array([[-1, -1,  1,  1]])

array([[0.4, 0.9, 0.2, 0.5],
       [0.2, 0.3, 0.6, 0.7]])

array([[-1, -1,  1,  1]])

In [5]:
x_L = [x_L_1, x_L_2, x_L_3]
x_L

[array([[0.1, 0.4, 0.3, 0.9],
        [0.5, 0.4, 0.8, 0.7]]),
 array([[0.1, 0.6, 0.2, 0.7],
        [0.3, 0.4, 0.8, 0.6]]),
 array([[0.4, 0.9, 0.2, 0.5],
        [0.2, 0.3, 0.6, 0.7]])]

In [6]:
y_L = [y_L_1, y_L_2, y_L_3]
y_L

[array([[-1, -1,  1,  1]]),
 array([[-1, -1,  1,  1]]),
 array([[-1, -1,  1,  1]])]

In [7]:
x_U_1 = U
x_U_2 = U
x_U_3 = U

x_U = [x_U_1, x_U_2, x_U_3]

display(x_U)

[array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])]

In [8]:
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])
x_U_tmp

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

In [9]:
x_S_1 = np.hstack([x_U_1, x_L_1])
x_S_2 = np.hstack([x_U_2, x_L_2])
x_S_3 = np.hstack([x_U_3, x_L_3])

x_S = [x_S_1, x_S_2, x_S_3]

x_S

[array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.4, 0.3, 0.9],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.4, 0.8, 0.7]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.6, 0.2, 0.7],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.3, 0.4, 0.8, 0.6]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.4, 0.9, 0.2, 0.5],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.2, 0.3, 0.6, 0.7]])]

In [10]:
# 6 x 12 の行列
M_1 = np.array([[-1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1]])

M_2 = np.array([[-1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1]])

M = [M_1, M_2]
print(M)

[array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]]), array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]])]


In [11]:
# 6 x 1 の行列（列ベクトル）
q_1 = np.array([[1, 1, 1, 1, 1, 1]]).T
q_2 = np.array([[1, 1, 1, 1, 1, 1]]).T

q = [q_1, q_2]
print(q)

[array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]]), array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]])]


## 1.2 目的関数の実装

メモ

* cp.quad_form で計算できるように，シグマ記号を二次形式で書き換える

In [14]:
# 通常の内積

def kernel_function(x1, x2):
    return x1.T @ x2

k = kernel_function


# 変数 
lambda_j_l = cp.Variable(shape=(3, 4), nonneg=True) # j \in {1, 2, 3}, l \in {1, 2, 3, 4}
lambda_h_i = cp.Variable(shape=(2, 6), nonneg=True) # h \in {1, 2}, i \in {1, 2, 3, 4, 5, 6}
eta_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
eta_hat_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

lambda_j_l, lambda_h_i, eta_j_s, eta_hat_j_s

(Variable((3, 4), nonneg=True),
 Variable((2, 6), nonneg=True),
 Variable((3, 10), nonneg=True),
 Variable((3, 10), nonneg=True))

In [15]:
len_j = 3
len_l = 4 

len_h = 2
len_I_h = 6

len_u = 12

len_s = 10

# とりあえずこれで
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])
x_U_tmp

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

### 1.2.1 目的関数

In [86]:
obj_func = 0


for j in range(len_j):
    A = np.zeros((len_l, len_l))
    for l in range(len_l):
        for l_prime in range(len_l):
            A[l, l_prime] = y_L[j][0, l] * y_L[j][0, l_prime] * k(x_L[j][:, l], x_L[j][:, l_prime])
    # print(f'A \n{A}\n')

    obj_func += 4 * cp.quad_form(lambda_j_l[0, :], A)


    # tmp_obj = 0
    # for h in range(len_h):
    #     for h_prime in range(len_h):
    #         # 各 h について，I_h （h が取る要素の数）がそれぞれ違えば，ここは変更しないといけない
    #         B = np.zeros((len_I_h, len_I_h))
    #         for i in range(len_I_h):
    #             for i_prime in range(len_I_h):
    #                 for u in range(len_u):
    #                     for u_prime in range(len_u):
    #                         B[i, i_prime] = M[h][i, u] * M[h_prime][i_prime, u_prime] * k(x_U_tmp[:, u], x_U_tmp[:, u_prime])
    #         tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), B])
    #         tmp2 = np.hstack([B.T, np.zeros((len_I_h, len_I_h))])
    #         B = np.vstack([tmp1, tmp2])
    #         tmp_vars = cp.hstack([lambda_h_i[h], lambda_h_i[h_prime]])
    #         cp.quad_form(tmp_vars, B)
    #         # tmp_obj += cp.quad_form(tmp_vars, B)
    #         tmp_obj += cp.quad_form(tmp_vars, B, assume_PSD=True)
    
    # obj_func += tmp_obj


    D = np.zeros((len_s, len_s))
    for s in range(len_s):
        for s_prime in range(len_s):
            D[s, s_prime] = k(x_S[j][:, s], x_S[j][:, s_prime])
    # print(f'D \n{D}\n')
    
    obj_func += cp.quad_form(eta_j_s[j, :] - eta_hat_j_s[j, :], D)


    tmp_obj = 0
    for h in range(len_h):
        E = np.zeros((len_l, len_I_h))
        for l in range(len_l):
            for i in range(len_I_h):
                tmp = 0
                for u in range(len_u):
                    tmp += y_L[j][0, l] * M[h][i, u] * k(x_L[j][:, l], x_U_tmp[:, u])
                E[l, i] = tmp
        tmp1 = np.hstack([np.zeros((len_l, len_l)), E])
        tmp2 = np.hstack([E.T, np.zeros((len_I_h, len_I_h))])
        E = np.vstack([tmp1, tmp2])
        tmp_vars = cp.hstack([lambda_j_l[j], lambda_h_i[h]])
        # cp.quad_form(tmp_vars, E)
        tmp_obj += cp.quad_form(tmp_vars, E, assume_PSD=True)
    
    # ここで -4 でかけると 全体で concavity がなくなる
    # obj_func += (-4) * tmp_obj
    obj_func += 4 * tmp_obj


    F = np.zeros((len_l, len_s))
    for l in range(len_l):
        for s in range(len_s):
            F[l, s] = y_L[j][0, l] * k(x_L[j][:, l], x_S[j][:, s])
    tmp1 = np.hstack([np.zeros((len_l, len_l)), F])
    tmp2 = np.hstack([F.T, np.zeros((len_s, len_s))])
    F = np.vstack([tmp1, tmp2])
    tmp_vars = cp.hstack([lambda_j_l[j], eta_j_s[j] - eta_hat_j_s[j]])

    # obj_func += 4 * cp.quad_form(tmp_vars, F)
    obj_func += 4 * cp.quad_form(tmp_vars, F, assume_PSD=True)
    

    tmp_obj = 0
    for h in range(len_h):
        G = np.zeros((len_I_h, len_s))
        for i in range(len_I_h):
            for s in range(len_s):
                tmp = 0
                for u in range(len_u):
                    tmp += M[h][i, u] * k(x_U_tmp[:, u], x_S[j][:, s])
                G[i, s] = tmp
        tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), G])
        tmp2 = np.hstack([G.T, np.zeros((len_s, len_s))])
        G = np.vstack([tmp1, tmp2])
        tmp_vars = cp.hstack([lambda_h_i[h], eta_j_s[j] - eta_hat_j_s[j]])
        # cp.quad_form(tmp_vars, G)
        tmp_obj += cp.quad_form(tmp_vars, G, assume_PSD=True)

    # ここで，-2 でかけると，全体で concavity がなくなる
    # obj_func += (-2) * tmp_obj
    obj_func += 2 * tmp_obj


for j in range(len_j):
    for l in range(len_l):
        obj_func += lambda_j_l[j, l]

for h in range(len_h):
    for i in range(len_I_h):
        tmp = 0
        for u in range(len_u):
            tmp += M[h][i, u]
        
        obj_func += lambda_h_i[h, i] * (1/2 * tmp + q[h][i,0])

tmp = 0
for j in range(len_j):
    for s in range(len_s):
        tmp += eta_j_s[j, s] + eta_hat_j_s[j, s]
obj_func += -1/2 * tmp


obj_func = (-1/2) * obj_func

display(obj_func)

Expression(CONCAVE, UNKNOWN, (1, 1))

In [87]:
print(obj_func)

-0.5 @ (4.0 @ QuadForm(var1[0, 0:4], [[ 0.26  0.24 -0.43 -0.44]
 [ 0.24  0.32 -0.44 -0.64]
 [-0.43 -0.44  0.73  0.83]
 [-0.44 -0.64  0.83  1.3 ]]) + QuadForm(var3[0, 0:10] + -var4[0, 0:10], [[0.26 0.38 0.25 0.23 0.19 0.35 0.26 0.24 0.43 0.44]
 [0.38 0.58 0.43 0.45 0.41 0.65 0.38 0.4  0.65 0.76]
 [0.25 0.43 0.41 0.52 0.53 0.7  0.25 0.36 0.47 0.73]
 [0.23 0.45 0.52 0.73 0.78 0.95 0.23 0.44 0.48 0.93]
 [0.19 0.41 0.53 0.78 0.85 1.   0.19 0.44 0.43 0.95]
 [0.35 0.65 0.7  0.95 1.   1.25 0.35 0.6  0.7  1.25]
 [0.26 0.38 0.25 0.23 0.19 0.35 0.26 0.24 0.43 0.44]
 [0.24 0.4  0.36 0.44 0.44 0.6  0.24 0.32 0.44 0.64]
 [0.43 0.65 0.47 0.48 0.43 0.7  0.43 0.44 0.73 0.83]
 [0.44 0.76 0.73 0.93 0.95 1.25 0.44 0.64 0.83 1.3 ]]) + 4.0 @ (QuadForm(Hstack(var1[0, 0:4], var2[0, 0:6]), psd_wrap([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 

In [88]:
constraint_1 = 0

for j in range(len_j):
    for h in range(len_h):
        for i in range(len_I_h):
            for u in range(len_u):
                constraint_1 += lambda_h_i[h, i] * M[h][i, u]

    for l in range(len_l):
        constraint_1 += (-2) * lambda_j_l[j, l] * y_L[j][0, l]

    for s in range(len_s):
        constraint_1 += eta_j_s[j, s] - eta_hat_j_s[j, s]

# display(constraint_1)


C_1, C_2 = 2.5, 2.5

objective = cp.Maximize(obj_func)

constraints = [
    constraint_1 == 0,
    lambda_j_l >=0,
    lambda_j_l <= C_1,
    lambda_h_i >= 0,
    lambda_h_i <= C_2,
    eta_j_s >=0,
    eta_hat_j_s >= 0
]


problem = cp.Problem(objective, constraints)
result = problem.solve()
print(result)

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


SolverError: Workspace allocation error!

In [81]:
lambda_h_i.value, lambda_j_l.value, eta_hat_j_s.value, eta_j_s.value

(None, None, None, None)

In [50]:
cp.settings.SOLVERS

['CLARABEL',
 'ECOS',
 'CVXOPT',
 'GLOP',
 'GLPK',
 'GLPK_MI',
 'SCS',
 'SDPA',
 'GUROBI',
 'OSQP',
 'CPLEX',
 'MOSEK',
 'CBC',
 'COPT',
 'XPRESS',
 'PROXQP',
 'NAG',
 'PDLP',
 'SCIP',
 'SCIPY']

In [93]:
# Define the data
t = 3  # number of tasks
v = 2  # number of logical constraints considered
ns = 0  # number of pointwise constraints

# Define L1, L2, L3, and U as lists of tuples
L1 = [(0.1, 0.5, -1), (0.4, 0.4, -1), (0.3, 0.8, 1), (0.9, 0.7, 1)]
L2 = [(0.1, 0.3, -1), (0.6, 0.4, -1), (0.2, 0.8, 1), (0.7, 0.6, 1)]
L3 = [(0.4, 0.2, -1), (0.9, 0.3, -1), (0.2, 0.6, 1), (0.5, 0.7, 1)]
U = [(0.1, 0.5), (0.3, 0.7), (0.5, 0.4), (0.8, 0.3), (0.9, 0.2), (1, 0.5)]

# Create numpy arrays for L and S
L = np.zeros((2, len(L1), t))
S = np.zeros((2, len(L1) + len(U), t))

for i, L_i in enumerate([L1, L2, L3]):
    L[:, :, i] = np.array(L_i).T
    # if v != 0:
    #     S[:, :, i] = np.hstack((L[:, :, i], np.array(U).T))
    # else:
    #     S[:, :, i] = L[:, :, i]

ValueError: could not broadcast input array from shape (3,4) into shape (2,4)

In [89]:

l = np.zeros(t)
ns = 0
s = np.zeros(t)

for i in range(t):
    if v != 0:
        u = len(U)
    else:
        u = 0
    l[i] = L.shape[1]
    ns += l[i]
    s[i] = S.shape[1]

c1 = 2.5  # degree of satisfaction for pointwise slacks
c2 = 2.5  # degree of satisfaction for logical slacks

lb = np.zeros(3 * t + ns + v)
ub = np.zeros(3 * t + ns + v)

for i in range(3 * t):
    lb[i] = -100000
    ub[i] = 100000

for i in range(3 * t, 3 * t + ns + v):
    lb[i] = 0
    ub[i] = 100000

H = np.zeros((3 * t + ns + v, 3 * t + ns + v))

for i in range(3 * t):
    if i % 3 != 0:
        H[i, i] = 1

f = np.zeros(3 * t + ns + v)

for i in range(3 * t, 3 * t + ns):
    f[i] = c1

for i in range(3 * t + ns, 3 * t + ns + v):
    f[i] = c2

A = np.zeros((0, 3 * t + ns + v))
b = np.zeros(0)
count = 0

# POINTWISE CONSTRAINTS

for i in range(t):
    k = 3 * i
    for j in range(int(l[i])):
        A_j = np.zeros(3 * t + ns + v)
        A_j[k] = -2 * L[0, j, i] * L[2, j, i]
        A_j[k + 1] = -2 * L[1, j, i] * L[2, j, i]
        A_j[k + 2] = -2 * L[2, j, i]
        A_j[count + j + 3 * t] = -2
        b = np.append(b, -1 - L[2, j, i])
        A = np.vstack((A, A_j))
    count += int(l[i])

# CONCISTENCY CONSTRAINTS

if v > 0:
    for i in range(t):
        k = 3 * i
        for j in range(int(s[i])):
            A_j = np.zeros(3 * t + ns + v)
            A_j[k] = S[0, j, i]
            A_j[k + 1] = S[1, j, i]
            A_j[k + 2] = 1
            b = np.append(b, 1)
            A_j_s = np.zeros(3 * t + ns + v)
            A_j_s[k] = -S[0, j, i]
            A_j_s[k + 1] = -S[1, j, i]
            A_j_s[k + 2] = -1
            b = np.append(b, 0)
            A = np.vstack((A, A_j, A_j_s))
        count += 2 * int(s[i])

# LOGICAL CONSTRAINTS

if v > 0:
    for i in range(2):
        k = 3 * i
        for j in range(u):
            A_j = np.zeros(3 * t + ns + v)
            A_j[k] = U[j][0]
            A_j[k + 1] = U[j][1]
            A_j[k + 2] = 1
            A_j[k + 3] = -U[j][0]
            A_j[k + 4] = -U[j][1]
            A_j[k + 5] = -1
            A_j[3 * t + ns + i] = -1
            b = np.append(b, 0)
            A = np.vstack((A, A_j))

# OPTIMAL SOLUTION
x = cp.Variable(3 * t + ns + v)
objective = cp.Minimize(0.5 * cp.quad_form(x, H) + f @ x)
constraints = [A @ x <= b]
problem = cp.Problem(objective, constraints)
problem.solve()

print("Optimal solution:")
print(x.value)


ValueError: could not broadcast input array from shape (3,4) into shape (2,4)